
# POC 激活验证

In [20]:
import dgl
import torch
from utils_gnn import cnn2graph_activation
# from model_lib import mnist_cnn_model as father_model
from utils_basic import load_spec_model
from utils_gnn import padding, unpadding
from dgl.data import DGLDataset
import os
from tqdm import tqdm
from dgl import save_graphs, load_graphs
import torch
import json
from torch import nn
import numpy as np
# from model_lib.mnist_cnn_model import Model
from random import randint
from utils_basic import load_spec_model
from sklearn.metrics import roc_auc_score
from torchinfo import summary
import torch.nn.functional as F
from dgl.nn.pytorch.conv import GINConv
from dgl.nn.pytorch.glob import SumPooling, AvgPooling, MaxPooling, SortPooling
from utils_gnn import MLP


# x = '/home/dorian/repos/Meta-Nerual-Trojan-Detection/shadow_model_ckpt/mnist/models5/shadow_jumbo_9.model'
x = './shadow_model_ckpt/mnist/models5/shadow_jumbo_0.model'
# load model 
# Model = load_spec_model(father_model, '5')
from model_lib.mnist_cnn_model import Model6 as Model
model = Model(gpu=True)
params = torch.load(x)
model.load_state_dict(params)
del params

# load model detail 
model_detail = {}
model_detail_path = "./intermediate_data/model_detail.json"
import json
with open(model_detail_path, 'r') as f:
    model_detail = json.load(f)
# print(model_detail)
g = cnn2graph_activation(model, model_detail['mnist']['5'])
dgl.save_graphs('./intermediate_data/grapj_test.bin', g)
del model_detail

import torch
import torchvision
import torchvision.transforms as transforms
# from utils_gnn import SGNACT
GPU = True
if GPU:
        torch.cuda.manual_seed_all(0)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        
transform = transforms.Compose([
            transforms.ToTensor(),
        ])
BATCH_SIZE = 1
# MNIST image dataset 
trainset = torchvision.datasets.MNIST(root='./raw_data/', train=True, download=True, transform=transform)
dataloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE)

# get a image
image = None
label = None
for i, (x_in, y_in) in enumerate(dataloader):
    image = x_in
    model(image)
    label = y_in
    break
del trainset, dataloader

In [21]:
# define the init process
cnt = 0 
def init_conv(data, data_size, weight, bias, kernel_size, stride, padding):
    global cnt
    row, col = weight.size()
    # print(weight.size())
    # get actual conv kernel weight and bias
    w = unpadding(weight, 1, kernel_size[0]*kernel_size[1])
    w = w.reshape(1, 1, kernel_size[0], kernel_size[1])
    ws = w
    np.savetxt("./intermediate_data/init/weight-%d.csv" % cnt, ws[0][0].cpu().numpy(), delimiter=',')
    cnt += 1
    b = unpadding(bias, 1, 1)[0]
    np.savetxt("./intermediate_data/init/bias-%d.csv" % cnt, b.cpu().numpy(), delimiter=',')
    # get conv operator 
    # print("kernel_size, stride, padding:")
    # print(kernel_size, stride, padding)
    operator = torch.nn.Conv2d(1, 1, kernel_size=kernel_size, 
                    stride=stride, padding=padding)
    # set conv operator weight and bias
    operator.weight.data = w
    operator.bias.data = b
    # conduct conv operation
    # print("conv input size:", data.size())
    x = operator(data.to("cuda"))

    return x


def initiate_node_feature(graph, image):
    ft = None 
    mask = graph.ndata['layer_idx'] == 0
    out_channels = int(sum(mask))
    ft = torch.zeros((len(graph.nodes()), 1, 1, 28, 28))
    convd_size = torch.zeros((len(graph.nodes()), 2))
    for i in range(out_channels):
        kernel_size, stride, padding = graph.ndata['kernel_params'][i]
        # do conv
        res_ft = init_conv(image, None, graph.ndata['kernel_weight'][i], graph.ndata['bias'][i],
                kernel_size, stride, padding)
        # do relu
        relu_opt = torch.nn.functional.relu
        res_ft = relu_opt(res_ft)

        # do max pooling
        pooling = graph.ndata['pooling_params'][i]
        if pooling.all() != 0:
            # do max_pooling
            kernel_size, stride, pad, dilation, ceil_mode = pooling
            max_pooling_operator = torch.nn.MaxPool2d(kernel_size=kernel_size, stride=stride, 
                padding=pad, dilation=dilation, ceil_mode=ceil_mode)
            
            res_ft = max_pooling_operator(res_ft)
        _, _, r, c = res_ft.size()
        ft[i] = res_ft
        convd_size[i] = torch.tensor([int(r), int(c)])
    graph.ndata['ft'] = ft.to("cuda")
    graph.ndata['ft_size'] = convd_size.to("cuda")



In [22]:
# init ft feature and get subg
with torch.no_grad():
    initiate_node_feature(g, image)
print(g.ndata['layer_type'])
# get node id
def neural_conv_nodes(nodes):
    return nodes.data['layer_type'] == 0
nodes_idx = g.filter_nodes(neural_conv_nodes)
# print(nodes_idx)
subg = dgl.node_subgraph(g, nodes_idx, relabel_nodes=True)
print(subg.nodes())
ft32 = None
# got the right subg of conv layers

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
       device='cuda:0')
tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
        54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,
        72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87],
       device='cuda:0')


In [23]:
from utils_gnn import equals, prepare_data
import pdb

def prepare_conv_data(nkernel_weight, 
                      nkernel_size, 
                      nkernel_bias, 
                      nkernel_params, 
                      channels):
    # prepare conv kernel weight
    _, width, height = nkernel_size
    kernel_weight = nkernel_weight
    kernel_weight = unpadding(kernel_weight, channels, width * height)
    kernel_weight = kernel_weight.reshape(1, channels, width, height)
    # prepare conv bias
    bias = nkernel_bias
    kernel_bias = unpadding(bias, 1, 1)[0]

    # prepare conv operator
    kernel_size, stride, pad = nkernel_params

    return kernel_weight, kernel_bias, kernel_size, stride, pad
    
def do_operation(data, conv_opt, pooling_params):
    # do conv
    torch.save(data, "./intermediate_data/reshaped_data.pt")
    conv_ft = conv_opt(data.to("cuda")).to("cuda")
    _, _, width, height = conv_ft.size()
    np.savetxt("./intermediate_data/process/conv_processed_ft.csv", conv_ft.reshape(width, height).cpu().numpy(), delimiter=',')
    # do relu
    relu_ft = torch.nn.functional.relu(conv_ft)
    np.savetxt("./intermediate_data/process/relu_processed_ft.csv", relu_ft.reshape(width, height).cpu().numpy(), delimiter=',')
    ret_ft = relu_ft
    # do pooling
    if pooling_params.any() != 0:
        kernel_size, stride, pad, dilation, ceil_mode = pooling_params
        kernel_size, stride, pad, dilation, ceil_mode = int(kernel_size), int(stride), int(pad), int(dilation), bool(ceil_mode)
        max_pooling_operator = torch.nn.MaxPool2d(kernel_size=kernel_size, stride=stride, 
            padding=pad, dilation=dilation, ceil_mode=ceil_mode)
        
        ret_ft = max_pooling_operator(relu_ft)
        _, _, width, height = ret_ft.size()
        # np.savetxt("./intermediate_data/ret_ft.csv", ret_ft.reshape(width, height).cpu().numpy(), delimiter=',')

    return ret_ft



def my_reduce(nodes):
    # received data and their size
    mfeat = nodes.mailbox['m']
    msize = nodes.mailbox['n']
    
    mf_size = mfeat.size()
    n_nodes = mf_size[0]
    n_channels = mf_size[1]

    # prepare ret_ft & ret_size, input size (1, 1, 28, 28)
    ret_ft = torch.zeros((n_nodes, 1, 1, 28, 28)).to("cuda")
    ret_size = torch.zeros((n_nodes, 2)).to("cuda")

    # for each node, do conv for received data
    for out_idx in range(n_nodes):

        # prepare received data and conv weight
        received_data = mfeat[out_idx]
        received_size = msize[out_idx]
        np.savetxt("./intermediate_data/process/rec-size-%d.csv" % out_idx, received_size.cpu().numpy(), delimiter=',')
        torch.save(received_data, "./intermediate_data/process/receivec_data.pt")

        # skip if all zeros
        if equals(received_data, zeros=True):
            continue
        # (n, 1, 1, 28, 28) -> (1, n, width, height)
        data = prepare_data(received_data, received_size)

        # prepare conv operator data
        kernel_weight, kernel_bias, kernel_size, stride, pad = prepare_conv_data(nodes.data['kernel_weight'][out_idx], 
                                                                                nodes.data['kernel_size'][out_idx],
                                                                                nodes.data['bias'][out_idx], 
                                                                                nodes.data['kernel_params'][out_idx], 
                                                                                n_channels)
        
        # prepare conv operator
        conv_opt = torch.nn.Conv2d(n_channels, 1, kernel_size=kernel_size, stride=stride, padding=pad)
        conv_opt.weight.data = kernel_weight
        conv_opt.bias.data = kernel_bias
        torch.save(kernel_weight, "./intermediate_data/process/kernel_weight.pt")
        np.savetxt("./intermediate_data/process/kernel-bias-%d.csv" % (out_idx),  kernel_bias.cpu().numpy(), delimiter=',')
        # print("kernel weight size:", kernel_weight.size(), kernel_size)
        # print("kernel_bias size:", kernel_size.size())

        # do operation
        ft = do_operation(data, conv_opt, nodes.data['pooling_params'][out_idx])

        # save ft size
        _, _, width, height = ft.size()
        np.savetxt("./intermediate_data/process/ret_ft.csv", ft.reshape(width, height).cpu().numpy(), delimiter=',')
        pad_ft = padding(ft.reshape(width, height), 28, 28)
        np.savetxt("./intermediate_data/process/pad_ft.csv", pad_ft.cpu().numpy(), delimiter=',')
        reshaped_ft = pad_ft.reshape(1, 1, 28, 28)
        # update return data
        ret_ft[out_idx] = reshaped_ft
        ret_size[out_idx] = torch.tensor([width, height]).to("cuda")
        # a_break = input("go next?")
        
    # return size is [n, 1, 1, 28, 28], reduced from [n, m, 1, 1, 28, 28]
    return {'h': ret_ft, 'g': ret_size}

def my_message(edges):
    return {'m': edges.src['ft'], 'n': edges.src['ft_size']}



with torch.no_grad():
    # 当最后一个节点尚未传播到的时候，继续执行传播操作
    while equals(subg.ndata['ft_size'][-1], zeros=True):
        subg.update_all(my_message, my_reduce)
        for idx in range(len(subg.ndata['ft'])):
            if equals(subg.ndata['ft'][idx], zeros=True):
                subg.ndata['ft'][idx] = subg.ndata['h'][idx]
            if equals(subg.ndata['ft_size'][idx], zeros=True):
                subg.ndata['ft_size'][idx] = subg.ndata['g'][idx]



In [24]:
print(subg.ndata['ft'].size())
print(subg.ndata['ft_size'].size())

torch.Size([88, 1, 1, 28, 28])
torch.Size([88, 2])


In [25]:
def print_zeros_in_feat(key):
    print(key + ": ")
    zeros = []
    nonzero = []
    for id in subg.nodes():
        id = int(id)
        feat = subg.nodes[id].data[key]
        if torch.eq(feat, torch.zeros(feat.size()).to("cuda")).all() == True:
            zeros.append(id)
        else:
            nonzero.append(id)
    print("zeros - ", zeros)
    print("nonzeros - ", nonzero)

print_zeros_in_feat('ft')
print_zeros_in_feat('h')
print_zeros_in_feat('ft_size')
print_zeros_in_feat('g')

ft: 
zeros -  [46, 47, 49, 57, 65, 72, 73, 75]
nonzeros -  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 48, 50, 51, 52, 53, 54, 55, 56, 58, 59, 60, 61, 62, 63, 64, 66, 67, 68, 69, 70, 71, 74, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87]
h: 
zeros -  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 46, 47, 49, 57, 65, 72, 73, 75]
nonzeros -  [32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 48, 50, 51, 52, 53, 54, 55, 56, 58, 59, 60, 61, 62, 63, 64, 66, 67, 68, 69, 70, 71, 74, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87]
ft_size: 
zeros -  []
nonzeros -  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 

In [26]:
size = subg.ndata['ft_size'][87]
w, h = size

w, h = int(w), int(h)

print(size, w, h)

res = unpadding(subg.ndata['ft'][85].reshape(28, 28), w, h)
print(res)

tensor([7., 7.], device='cuda:0') 7 7
tensor([[ 1.7792,  0.0000,  0.0000,  0.0000,  0.0000,  1.1611,  6.6959],
        [ 5.1518,  1.1776,  0.0000,  3.3811,  9.1028,  5.7674,  7.4037],
        [ 5.1297,  6.2894,  0.0000, 12.7424, 16.8882,  6.2509,  0.0445],
        [ 1.4539,  8.4813,  6.2850,  0.0000,  8.7977, 20.9802,  0.1473],
        [ 1.3156,  0.3564,  0.0000,  0.0000,  9.1116, 21.8856,  0.0572],
        [ 2.9410,  0.0000,  2.2457, 11.6646,  9.9770,  7.0505,  0.0000],
        [ 4.3832,  0.1667,  6.2607,  7.3015,  0.0000,  0.0000,  0.0000]],
       device='cuda:0')


In [27]:
cnt = 0
ft_list = []
for f,s in zip(subg.ndata['ft'], subg.ndata['ft_size']):
    w, h = s
    w, h = int(w), int(h)
    data = unpadding(f.reshape(28, 28), w, h)
    # print(w, h, cnt)
    ft_list.append(data)
    np.savetxt("./intermediate_data/graph_results/node-%d.csv" % cnt, data.cpu(), delimiter=',')
    cnt += 1

cnt = 0
cnn_params = None 
with open("./intermediate_data/params.json", "r") as f:
    cnn_params = json.load(f)
from torch.nn.functional import relu
from utils_gnn import unpadding
conv1 = torch.tensor(cnn_params['conv1'])
conv2 = torch.tensor(cnn_params['conv2'])
conv3 = torch.tensor(cnn_params['conv3'])
conv4 = torch.tensor(cnn_params['conv4'])
l = [conv1, conv2, conv3, conv4]
neural_list = []
for layer in l:
    for neural in layer[0]:
        neural_list.append(neural)
        np.savetxt("./intermediate_data/model_results/node-%d.csv" % cnt, neural.cpu(), delimiter=',')
        cnt += 1

In [28]:
# add message transmitted ndata to original graph 
g.ndata['ft'][:len(subg.ndata['ft'])] = subg.ndata['ft']
g.ndata['ft_size'][:len(subg.ndata['ft_size'])] = subg.ndata['ft_size']
print(unpadding(g.ndata['ft'][87].reshape(28, 28), 7, 7))

tensor([[ 0.0000,  0.9805,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000, 16.5474, 34.7467, 25.6497, 23.0430, 23.0015, 12.2277],
        [ 0.0000,  2.5368, 39.6762, 26.9853,  3.7355,  4.5688,  4.2940],
        [ 0.0000,  0.0000,  4.3969, 28.0040, 25.0063,  0.0000,  0.2286],
        [ 0.0000,  0.0000,  0.0000, 11.7499, 20.8333, 16.8679,  1.9555],
        [ 8.7424, 24.6951, 27.1628, 18.3077, 18.5936, 10.5977,  0.9699],
        [ 0.0000, 10.9583, 13.3883,  5.1577,  0.0000,  0.0000,  0.2703]],
       device='cuda:0')


In [29]:
# split full connected neurals
def neural_fc_nodes(nodes):
    layer_mask = nodes.data['layer_type'] == 1
    layer_idxs = nodes.data['layer_idx'][layer_mask]
    n = layer_idxs[0] 
    return nodes.data['layer_idx'] >= n
fc_nodes = g.filter_nodes(neural_fc_nodes)
print(fc_nodes)
print(g.ndata['layer_idx'])
print(g.ndata['layer_type'])
subfcg = dgl.node_subgraph(g, fc_nodes, relabel_nodes=True)

tensor([88, 89, 90], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 5, 6],
       device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
       device='cuda:0')


In [30]:
def encode_fc_feat(data):
    # data shape = (1, n)
    data = padding(data, 1, 23 * 23)
    ret = torch.zeros([23, 1, 23]).to("cuda")
    start = 0
    end = 23
    idx = 0
    while end <= 23 * 23:
        cur_data = data[0][start:end]
        ret[idx] = cur_data.view(1, 23)
        idx += 1
        start += 23 
        end += 23
    # reshape (23, 1, 23) to (1, 1, 28, 28)
    ret = padding(ret.reshape(23, 23), 28, 28)
    return ret.reshape(1, 1, 28, 28)

def  decode_fc_feat(data, n):
    # data shape = (1, 1, 28, 28)
    ret = unpadding(data.reshape(28, 28), 23, 23) # (23, 23)
    ret = torch.cat([ret[i] for i in range(len(ret))]) # (23*23)
    ret = ret.reshape(1, 23*23)
    ret = unpadding(ret, 1, n)
    return ret


In [35]:

from torch.nn.parameter import Parameter
# get conv feat before first fc layer 
def get_fc_first_ft(g):
    def neural_nodes_before_fc(nodes):
        layer_mask = nodes.data['layer_type'] == 1
        layer_idxs = nodes.data['layer_idx'][layer_mask]
        n = layer_idxs[0] 
        return nodes.data['layer_idx'] == n - 1 
    pre_nodes_idx = g.filter_nodes(neural_nodes_before_fc)
    # print(pre_nodes_idx)
    # get feats and reshape to (1, channels, width, height)
    prefg = dgl.node_subgraph(g, pre_nodes_idx, relabel_nodes=True)
    pre_feats = prefg.ndata['ft']
    # (8, 1, 1, 28, 28)
    # print(pre_feats.shape)
    n_pre_nodes = len(pre_nodes_idx)
    shape = prefg.ndata['ft_size'][0]
    w, h = shape
    w, h = int(w), int(h)
    del prefg
    # reshape to (1, 8, 7, 7)
    feats = torch.zeros([n_pre_nodes, w, h])
    for idx in range(n_pre_nodes):
        cur_data = pre_feats[idx]
        cur_data = cur_data.reshape(28, 28)
        res_data = unpadding(cur_data, w, h)
        # print("res_data size:", res_data.size())
        feats[idx] = res_data
    data = feats
    # print("data size: ", data.shape)

    # concat weight
    def neural_concat_nodes(nodes):
        layer_mask = nodes.data['layer_type'] == 1
        layer_idxs = nodes.data['layer_idx'][layer_mask]
        n = layer_idxs[0] 
        return nodes.data['layer_idx'] == n
    n = g.filter_nodes(neural_concat_nodes)
    n = int(n)
    # print(n, g.ndata['kernel_size'][n])
    in_dim = g.ndata['kernel_size'][n][1]
    out_dim = g.ndata['kernel_size'][n][2]
    in_dim, out_dim = int(in_dim), int(out_dim)
    # print(n, " in_dim:", in_dim, " out_dim:", out_dim)
    fc_concat_weight = g.ndata['kernel_weight'][n]
    # reshape fc weight to (in_dim, out_dim)
    fc_concat_weight = unpadding(fc_concat_weight, in_dim, out_dim).t()
    # bias
    fc_concat_bias = g.ndata['bias'][n]
    bias_size = g.ndata['bias_size'][n]
    # print(bias_size)
    r, c = bias_size
    r, c = int(r), int(c)
    # reshape
    # print(fc_concat_bias.shape)
    fc_concat_bias = unpadding(fc_concat_bias, r, c)
    # init fc layer
    # print("Linear params: ", in_dim, out_dim)
    concat_opt = torch.nn.Linear(in_dim, out_dim)
    # print(concat_opt.weight.data.shape, fc_concat_weight.shape)
    concat_opt.weight = Parameter(fc_concat_weight)
    concat_opt.bias.data = Parameter(fc_concat_bias.reshape(512))

    # do concat
    # print("data size: ", data.shape, data.view(1, in_dim).shape)
    fc_concat_result = torch.nn.functional.relu(concat_opt(data.view(1, in_dim).to("cuda")))
    shape = fc_concat_result.size()
    r, c = shape
    r, c = int(r), int(c)
    return n, fc_concat_result, r, c

n, fc_concat_result, r, c = get_fc_first_ft(g)
# set init fc feat
g.ndata['ft'][n] = encode_fc_feat(fc_concat_result)
g.ndata['ft_size'][n] = torch.tensor([r, c], dtype=torch.float32).to("cuda")

Linear params:  392 512
torch.Size([512, 392]) torch.Size([512, 392])
data size:  torch.Size([8, 7, 7]) torch.Size([1, 392])


In [36]:
def do_linear_opt(nodes, idx, data, size):
    # prepare weight
    in_dim = nodes.data['kernel_size'][idx][1]
    out_dim = nodes.data['kernel_size'][idx][2]
    in_dim, out_dim = int(in_dim), int(out_dim)
    weight = nodes.data['kernel_weight'][idx]
    # (out_dim, in_dim) for calculation optimization
    weight = unpadding(weight, in_dim, out_dim).t()
    # prepare bias
    bias_size = nodes.data['bias_size'][idx]
    r, c = bias_size
    r, c = int(r), int(c)
    bias = nodes.data['bias'][idx] # (1, 512) -> (r, c)
    bias = unpadding(bias, r, c)
    # prepare Linear layer
    fc_opt = torch.nn.Linear(in_dim, out_dim)
    fc_opt.weight = Parameter(weight)
    fc_opt.bias = Parameter(bias.reshape(c))
    # prepare data (1, 1, 28, 28) -> (1, n)
    _, n = size[0]
    data = decode_fc_feat(data, n)

    # do Linear operation
    ret = torch.nn.functional.relu(fc_opt(data.view(1, in_dim).to("cuda")))
    return ret


def my_fc_message(edges):
    return {'mf': edges.src['ft'], 'ms': edges.src['ft_size']}
def my_fc_reduce(nodes):
    mfeat = nodes.mailbox['mf']
    msize = nodes.mailbox['ms']
    
    n_nodes = len(mfeat)
    ret_ft = torch.zeros((n_nodes, 1, 1, 28, 28)).to("cuda")
    ret_size = torch.zeros((n_nodes, 2)).to("cuda") 

    for out_idx in range(n_nodes):
        received_data = mfeat[out_idx]
        received_size = msize[out_idx]

        # skip if all zeros, that is not transformed
        if equals(received_size, zeros=True):
            continue
        # do Linear opt 
        ret = do_linear_opt(nodes, out_idx, received_data, received_size)
        np.savetxt("./intermediate_data/process/ret_fc_ft-%d.csv" % out_idx, ret.cpu().numpy(), delimiter=',')

        shape = ret.size()
        r, c = shape
        r, c = int(r), int(c)

        # set ret feat
        ret_ft[out_idx] = encode_fc_feat(ret)
        ret_size[out_idx] = torch.tensor([r, c], dtype=torch.float32).to("cuda")
    return {'h': ret_ft, 'g':ret_size}

In [37]:
with torch.no_grad():
    while equals(subfcg.ndata['ft_size'][-1], zeros=True):
        subfcg.update_all(my_fc_message, my_fc_reduce)
        for idx in range(len(subfcg.ndata['ft'])):
            if equals(subfcg.ndata['ft'][idx], zeros=True):
                subfcg.ndata['ft'][idx] = subfcg.ndata['h'][idx]
            if equals(subfcg.ndata['ft_size'][idx], zeros=True):
                subfcg.ndata['ft_size'][idx] = subfcg.ndata['g'][idx]

In [38]:
def print_zeros_in_feat(key):
    print(key + ": ")
    zeros = []
    nonzero = []
    for id in subfcg.nodes():
        id = int(id)
        feat = subfcg.nodes[id].data[key]
        if torch.eq(feat, torch.zeros(feat.size()).to("cuda")).all() == True:
            zeros.append(id)
        else:
            nonzero.append(id)
    print("zeros - ", zeros)
    print("nonzeros - ", nonzero)

print_zeros_in_feat('ft')
print_zeros_in_feat('h')
print_zeros_in_feat('ft_size')
print_zeros_in_feat('g')

ft: 
zeros -  []
nonzeros -  [0, 1, 2]
h: 
zeros -  [0]
nonzeros -  [1, 2]
ft_size: 
zeros -  []
nonzeros -  [0, 1, 2]
g: 
zeros -  [0]
nonzeros -  [1, 2]


In [39]:
import json
cnn_params = None 
with open("./intermediate_data/params.json", "r") as f:
    cnn_params = json.load(f)

fc1 = torch.tensor(cnn_params['fc1']).to("cuda")
fc2 = torch.tensor(cnn_params['fc2'])
print(fc1.shape, fc2.shape)

torch.Size([1, 512]) torch.Size([1, 128])


In [40]:
# compare fc1 out
print(data.shape, conv4.shape)
print("is input the same:", equals(data, conv4.reshape(8, 7, 7)))
print(concat_opt.weight.shape, model.fc1.weight.shape)
print("is the fc1 weight the same:", equals(concat_opt.weight, model.fc1.weight))
print((fc_concat_weight == model.fc1.weight).all() == True)
print(fc_concat_bias.shape, model.fc1.bias.shape)
print("is the fc1 bias the same:", equals(fc_concat_bias.reshape(512), model.fc1.bias))
print((fc_concat_bias == model.fc1.bias).all() == True)
print(fc_concat_result.shape, fc1.shape)
print("is the output the same:", equals(fc_concat_result, fc1))
print((fc_concat_result == fc1).all() == True)

torch.Size([7, 7]) torch.Size([1, 8, 7, 7])


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [41]:
# compare fc2 out
data = subfcg.ndata['ft'][-2]
_, n = subfcg.ndata['ft_size'][-2]
data = decode_fc_feat(data, n)
print(data.shape, fc2.shape)
print("is fc2 out the same:", equals(data.cpu(), fc2, precision=1e-5))


torch.Size([1, 128]) torch.Size([1, 128])
is fc2 out the same: True


In [42]:
# compare output layer out
data = subfcg.ndata['ft'][-1]
_, n = subfcg.ndata['ft_size'][-1]
data = decode_fc_feat(data, n)
ret, _ = model(image)
print(data.shape, ret.shape)
print("is output layer out the same:", equals(data, torch.nn.functional.relu(ret), precision=1e-4))

torch.Size([1, 10]) torch.Size([1, 10])
is output layer out the same: True


In [43]:
g.ndata['ft'][-len(subfcg.ndata['ft']):] = subfcg.ndata['ft']
g.ndata['ft_size'][-len(subfcg.ndata['ft_size']):] = subfcg.ndata['ft_size']

In [44]:
for idx in range(len(g.ndata['ft'])):
    g_ft = g.ndata['ft'][idx]
    g_ft_size = g.ndata['ft_size'][idx]
    if idx < len(subg.ndata['ft']):
       m_ft = subg.ndata['ft'][idx]
       m_ft_size = subg.ndata['ft_size'][idx]
    else:
       m_ft = subfcg.ndata['ft'][idx-len(subg.ndata['ft'])]
       m_ft_size = subfcg.ndata['ft_size'][idx-len(subg.ndata['ft'])]
    if not equals(g_ft, m_ft):
        print("feature not equals")
    if not equals(g_ft_size, m_ft_size):
        print("feature size not equals")
else:
    print("all equals")
        

all equals


In [45]:
print(data)

tensor([[ 0.0000,  0.0000,  2.1183, 18.8971,  7.8918, 11.7048,  0.0000,  0.0000,
          0.0000,  5.8714]], device='cuda:0')


In [46]:
print(torch.nn.functional.relu(ret))

tensor([[ 0.0000,  0.0000,  2.1183, 18.8971,  7.8918, 11.7048,  0.0000,  0.0000,
          0.0000,  5.8714]], device='cuda:0', grad_fn=<ReluBackward0>)


In [48]:
def activation_passing(img, g):
    # init ft feature on first conv layer and get conv subg
    with torch.no_grad():
        initiate_node_feature(g, img)
    
    def neural_conv_nodes(nodes):
        return nodes.data['layer_type'] == 0
    nodes_idx = g.filter_nodes(neural_conv_nodes)
    subg = dgl.node_subgraph(g, nodes_idx, relabel_nodes=True)
    # message passing on conv layers
    with torch.no_grad():
        # 当最后一个节点尚未传播到的时候，继续执行传播操作
        while equals(subg.ndata['ft_size'][-1], zeros=True):
            subg.update_all(my_message, my_reduce)
            for idx in range(len(subg.ndata['ft'])):
                if equals(subg.ndata['ft'][idx], zeros=True):
                    subg.ndata['ft'][idx] = subg.ndata['h'][idx]
                if equals(subg.ndata['ft_size'][idx], zeros=True):
                    subg.ndata['ft_size'][idx] = subg.ndata['g'][idx]
    # add message passing ndata on conv layers to original graph 
    g.ndata['ft'][:len(subg.ndata['ft'])] = subg.ndata['ft']
    g.ndata['ft_size'][:len(subg.ndata['ft_size'])] = subg.ndata['ft_size']
    # get fc subg 
    def neural_fc_nodes(nodes):
        layer_mask = nodes.data['layer_type'] == 1
        layer_idxs = nodes.data['layer_idx'][layer_mask]
        n = layer_idxs[0] 
        return nodes.data['layer_idx'] >= n
    fc_nodes = g.filter_nodes(neural_fc_nodes)
    subfcg = dgl.node_subgraph(g, fc_nodes, relabel_nodes=True)
    # init ft on first fc layer
    n, fc_concat_result, r, c = get_fc_first_ft(g)
    g.ndata['ft'][n] = encode_fc_feat(fc_concat_result)
    g.ndata['ft_size'][n] = torch.tensor([r, c], dtype=torch.float32).to("cuda")
    # message passing on fc layers 
    with torch.no_grad():
        while equals(subfcg.ndata['ft_size'][-1], zeros=True):
            subfcg.update_all(my_fc_message, my_fc_reduce)
            for idx in range(len(subfcg.ndata['ft'])):
                if equals(subfcg.ndata['ft'][idx], zeros=True):
                    subfcg.ndata['ft'][idx] = subfcg.ndata['h'][idx]
                if equals(subfcg.ndata['ft_size'][idx], zeros=True):
                    subfcg.ndata['ft_size'][idx] = subfcg.ndata['g'][idx]
    # add message passing ndata on fc layers to original graph
    g.ndata['ft'][-len(subfcg.ndata['ft']):] = subfcg.ndata['ft']
    g.ndata['ft_size'][-len(subfcg.ndata['ft_size']):] = subfcg.ndata['ft_size']
    return g

In [50]:
def test_activation_passing():
    def get_model():
        x = './shadow_model_ckpt/mnist/models5/shadow_jumbo_0.model'
        # load model 
        # Model = load_spec_model(father_model, '5')
        from model_lib.mnist_cnn_model import Model6 as Model
        model = Model(gpu=True)
        params = torch.load(x)
        model.load_state_dict(params)
        del params
        return model
    def get_graph():
        # load model detail 
        model_detail = {}
        model_detail_path = "./intermediate_data/model_detail.json"
        import json
        with open(model_detail_path, 'r') as f:
            model_detail = json.load(f)
        # print(model_detail)
        g = cnn2graph_activation(model, model_detail['mnist']['5'])
        dgl.save_graphs('./intermediate_data/grapj_test.bin', g)
        del model_detail
        return g 
    def get_image_dataset():
        # from utils_gnn import SGNACT
        GPU = True
        if GPU:
                torch.cuda.manual_seed_all(0)
                torch.backends.cudnn.deterministic = True
                torch.backends.cudnn.benchmark = False
                
        transform = transforms.Compose([
                    transforms.ToTensor(),
                ])
        BATCH_SIZE = 1
        # MNIST image dataset 
        trainset = torchvision.datasets.MNIST(root='./raw_data/', train=True, download=True, transform=transform)
        dataloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE)
        return dataloader
    model = get_model()
    g = get_graph()
    dataloader = get_image_dataset()
    for i, (x_in, y_in) in enumerate(dataloader):
        img = x_in
        label = y_in

        model_result, _ = model(img)
        passed_graph = activation_passing(img, g)
        graph_result = passed_graph.ndata['ft'][-1]
        _, n = passed_graph.ndata['ft_size'][-1]
        relu = torch.nn.functional.relu
        graph_result = relu(decode_fc_feat(graph_result, n))
        assert equals(model_result, graph_result), "output diffs: %d pic" % i
        if i > 100:
            break

test_activation_passing()

Linear params:  392 512
torch.Size([512, 392]) torch.Size([512, 392])
data size:  torch.Size([8, 7, 7]) torch.Size([1, 392])
torch.Size([1, 10]) torch.Size([1, 10])
0
Linear params:  392 512
torch.Size([512, 392]) torch.Size([512, 392])
data size:  torch.Size([8, 7, 7]) torch.Size([1, 392])
torch.Size([1, 10]) torch.Size([1, 10])
1


KeyboardInterrupt: 